<a href="https://colab.research.google.com/github/rishadb/sample_codes/blob/main/4RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from typing import Sequence
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


##HP
batch_size = 64
num_sequence = 28
input_size = 28
num_classes = 10
hidden_size = 256
learning_rate = .001
num_epoch = 2
num_layers = 2
transforms = transforms.ToTensor()

#DATA
train_dataset = datasets.MNIST(root = "datas/", transform=transforms, download=True)
test_dataset = datasets.MNIST(root = "datas/", train=False, transform=transforms,download = True )
train_dataloader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True, num_workers=2)
test_dataloader = DataLoader(dataset = test_dataset, batch_size = batch_size, num_workers=2)

#Defines
class BRNN(nn.Module):
  def __init__(self, input_features, num_classes, hidden_size, num_sequence):
    super().__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.LSTM = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True, bidirectional = True) #batch_first means batch_num comes in the shape, input = (bathc, seq, input_size)
    self.fc = nn.Linear(hidden_size*num_sequence*2, num_classes) #*2 since its BRNN, also, use only hidden_size if only takes last layer into account

  def forward(self, x):
    #h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # *2 only for BRNN
    #c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) #only for lstm
    out, _ = self.LSTM(x)#, (h0, c0)) # we get additional (h_n, c_n) of the last layer, out = (batch, seq, hidden)
    print(out.shape)
    #out = out[:, -1, :] #if we are considering only the last LsTM output
    out = out.reshape(out.shape[0], -1) #out was batch, seq, hidden_size
    #out = self.fc(out[:,-1,:]) # for taking only the last output into fc; all in batch, last line of outputs , num_sequences
    out = self.fc(out)
    print(f'n{out.shape}')
    return out

#initialize

model = BRNN(input_size, num_classes, hidden_size, num_sequence).to(device)
criterion = nn.CrossEntropyLoss()
optim = optim.RMSprop(model.parameters(),lr = learning_rate)

#training
for epoch in range(num_epoch):
  for i, (data, label) in enumerate(train_dataloader):
#original size = (batchsize, 1, features, sequence)

    data = data.to(device).squeeze(1) # RNN needs shape to be batch_size, input_features, num_sequence, but MNIST is bs, 1, h, w
    label = label.to(device)
    score = model(data)
    optim.zero_grad()
    loss = criterion(score, label)
    loss.backward()
    optim.step()

#accuracy check

def acc(model, loader):
  model.eval()
  with torch.no_grad():
    classes = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    n_samples = 0
    n_correct = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    for images, labels in loader:
      images = images.to(device).squeeze(1)
      labels = labels.to(device)

      outputs = model(images)

      _, predictions = outputs.max(1)
      n_samples += labels.size(0)
      n_correct += (predictions == labels).sum().item()
      for i in range(10):
        label = labels[i]
        pred = predictions[i]
        if label ==pred:
          n_class_correct[label] +=1
        n_class_samples[label] +=1

  acc = n_correct/ n_samples
  print(f"total accuracy is {acc * 100}%")
  class_acc = [n_class_correct[i]/ n_class_samples[i] for i in range(10)]

  for i in range(10):
    print(f"{classes[i]} accuracy: {class_acc[i] * 100}%")

  return acc

  acc(model, train_dataloader)
acc(model, test_dataloader)

SyntaxError: ignored

In [ ]:

from typing import Sequence
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


##HP
batch_size = 64
num_sequence = 28
input_size = 28
num_classes = 10
hidden_size = 256
learning_rate = .001
num_epoch = 2
num_layers = 2
transforms = transforms.ToTensor()

#DATA
train_dataset = datasets.MNIST(root = "datas/", transform=transforms, download=True)
test_dataset = datasets.MNIST(root = "datas/", train=False, transform=transforms,download = True )
train_dataloader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True, num_workers=2)
test_dataloader = DataLoader(dataset = test_dataset, batch_size = batch_size, num_workers=2)

#Defines
class BRNN(nn.Module):
  def __init__(self, input_features, num_classes, hidden_size, num_sequence):
    super().__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.LSTM = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True, bidirectional = True) #batch_first means batch_num comes in the shape, input = (bathc, seq, input_size)
    self.fc = nn.Linear(hidden_size*num_sequence*2, num_classes) #*2 since its BRNN, also, use only hidden_size if only takes last layer into account

  def forward(self, x):
    #h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # *2 only for BRNN
    #c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) #only for lstm
    out, _ = self.LSTM(x)#, (h0, c0)) # we get additional (h_n, c_n) of the last layer, out = (batch, seq, hidden)
    print(out.shape)
    #out = out[:, -1, :] #if we are considering only the last LsTM output
    out = out.reshape(out.shape[0], -1) #out was batch, seq, hidden_size
    #out = self.fc(out[:,-1,:]) # for taking only the last output into fc; all in batch, last line of outputs , num_sequences
    out = self.fc(out)
    print(f'n{out.shape}')
    return out

#initialize

model = BRNN(input_size, num_classes, hidden_size, num_sequence).to(device)
criterion = nn.CrossEntropyLoss()
optim = optim.RMSprop(model.parameters(),lr = learning_rate)

#training
for epoch in range(num_epoch):
  for i, (data, label) in enumerate(train_dataloader):
#original size = (batchsize, 1, features, sequence)

    data = data.to(device).squeeze(1) # RNN needs shape to be batch_size, input_features, num_sequence, but MNIST is bs, 1, h, w
    label = label.to(device)
    score = model(data)
    optim.zero_grad()
    loss = criterion(score, label)
    loss.backward()
    optim.step()

#accuracy check

def acc(model, loader):
  model.eval()
  with torch.no_grad():
    classes = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    n_samples = 0
    n_correct = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    for images, labels in loader:
      images = images.to(device).squeeze(1)
      labels = labels.to(device)

      outputs = model(images)

      _, predictions = outputs.max(1)
      n_samples += labels.size(0)
      n_correct += (predictions == labels).sum().item()
      for i in range(10):
        label = labels[i]
        pred = predictions[i]
        if label ==pred:
          n_class_correct[label] +=1
        n_class_samples[label] +=1

  acc = n_correct/ n_samples
  print(f"total accuracy is {acc * 100}%")
  class_acc = [n_class_correct[i]/ n_class_samples[i] for i in range(10)]

  for i in range(10):
    print(f"{classes[i]} accuracy: {class_acc[i] * 100}%")

  return acc

  acc(model, train_dataloader)
acc(model, test_dataloader)